In [ ]:
!pip install openimages
!pip install torch_snippets
!pip install pycocotools
!pip install git+git://github.com/waspinator/pycococreator.git@0.2.0

In [2]:
import numpy as np
from torch_snippets import *
import matplotlib.pyplot as plt
import pandas as pd
from pycococreatortools import pycococreatortools
from os import listdir
from os.path import isfile, join
from PIL import Image
import cv2
import torch

In [3]:
!wget -O train-annotations-object-segmentation.csv -q https://storage.googleapis.com/openimages/v5/train-annotations-object-segmentation.csv
!wget -O classes.csv -q https://raw.githubusercontent.com/openimages/dataset/master/dict.csv

In [4]:
required_classes = 'person,dog,bird,elephant,Zebra'
required_classes = [c.lower() for c in required_classes.lower().split(',')]
print(required_classes)

['person', 'dog', 'bird', 'elephant', 'zebra']


In [5]:
classes = pd.read_csv('classes.csv', header=None)
classes.columns = ['class','class_name']
classes = classes[classes['class_name'].map(lambda x: x in required_classes)]
print(classes)

           class class_name
224     /m/015p6       bird
753    /m/01g317     person
5916    /m/0898b      zebra
6420   /m/0bt9lr        dog
6432  /m/0bwd_0j   elephant


In [6]:
df=pd.read_csv('/content/train-annotations-object-segmentation.csv')
df.head(2)

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks
0,677c122b0eaa5d16_m04yx4_9a041d52.png,677c122b0eaa5d16,/m/04yx4,9a041d52,0.887500,0.960938,0.454167,0.720833,0.86864,0.95498 0.65197 1;0.89370 0.56579 1;0.94701 0....
1,05529ae018130c68_m09j2d_b1115fd0.png,05529ae018130c68,/m/09j2d,b1115fd0,0.086875,0.254375,0.504708,0.790960,0.80250,0.16388 0.50114 1;0.25069 0.75425 1;0.13478 0....


In [7]:
data = pd.merge(df, classes, left_on='LabelName', right_on='class')
data.head(2)

,MaskPath,ImageID,LabelName,BoxID,BoxXMin,BoxXMax,BoxYMin,BoxYMax,PredictedIoU,Clicks,class,class_name
0,e9a7da5ff1fd56df_m01g317_1803811b.png,e9a7da5ff1fd56df,/m/01g317,1803811b,0.468125,0.62875,0.188714,0.949121,0.86841,0.47513 0.38255 1;0.52118 0.91333 0;0.47513 0....,/m/01g317,person
1,18c1fd8342d55af8_m01g317_a9a97702.png,18c1fd8342d55af8,/m/01g317,a9a97702,0.394375,0.60500,0.241667,0.903333,0.81242,0.57036 0.81358 1;0.54599 0.65655 0;0.42280 0....,/m/01g317,person


In [9]:
subset_data = data.groupby('class_name').agg({'ImageID': lambda x: list(x)[:500]})
subset_data = flatten(subset_data.ImageID.tolist())
subset_data = data[data['ImageID'].map(lambda x: x in subset_data)]
subset_masks = subset_data['MaskPath'].tolist()

In [10]:
!mkdir -p masks
for c in Tqdm('0123456789abcdef'):
 !wget -q https://storage.googleapis.com/openimages/v5/train-masks/train-masks-{c}.zip
 !unzip -q train-masks-{c}.zip -d tmp_masks
 !rm train-masks-{c}.zip
 tmp_masks = Glob('tmp_masks', silent=True)
 items = [(m,fname(m)) for m in tmp_masks]
 items = [(i,j) for (i,j) in items if j in subset_masks]
 for i,j in items:
   os.rename(i, f'masks/{j}')
 !rm -rf tmp_masks

100%|██████████| 16/16 [21:47<00:00, 81.71s/it]


In [11]:
masks = Glob('masks')
masks = [fname(mask) for mask in masks]
subset_data = subset_data[subset_data['MaskPath'].map(lambda x: x in masks)]
subset_imageIds = subset_data['ImageID'].tolist()
from openimages.download import _download_images_by_id
!mkdir images 
_download_images_by_id(subset_imageIds, 'train', './images/')

2021-10-18 12:31:09.996 | INFO     | torch_snippets.paths:inner:24 - 6331 files found at masks
100%|██████████| 6331/6331 [04:54<00:00, 21.53it/s]


In [12]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [13]:
import zipfile
files = Glob('images') + Glob('masks') + ['train-annotations-object-segmentation.csv', 'classes.csv']
with zipfile.ZipFile('/content/drive/MyDrive/data.zip','w') as zipme:
    for file in Tqdm(files):
        zipme.write(file, compress_type=zipfile.ZIP_DEFLATED)

2021-10-18 12:37:05.661 | INFO     | torch_snippets.paths:inner:24 - 2204 files found at images
2021-10-18 12:37:05.702 | INFO     | torch_snippets.paths:inner:24 - 6331 files found at masks
100%|██████████| 8537/8537 [01:54<00:00, 74.58it/s]


In [14]:
!mkdir -p train/
!mv images train/myData2020
!mv masks train/annotations

## Preprocessing the data

In [31]:
import datetime

In [32]:
INFO = {
 "description": "MyData2021",
 "url": "None",
 "version": "1.0",
 "year": 2021,
 "contributor": "Rohit_Mondal",
 "date_created": datetime.datetime.utcnow().isoformat(' ')
}

In [33]:
LICENSES = [
    {
        "id": 1,
        "name": "HITK"
    }
]

In [34]:
CATEGORIES = [{'id': id+1, 'name': name.replace('/',''), 'supercategory': 'none'} for id,(_,(name, clss_name)) in enumerate(classes.iterrows())]
print(CATEGORIES)

[{'id': 1, 'name': 'm015p6', 'supercategory': 'none'}, {'id': 2, 'name': 'm01g317', 'supercategory': 'none'}, {'id': 3, 'name': 'm0898b', 'supercategory': 'none'}, {'id': 4, 'name': 'm0bt9lr', 'supercategory': 'none'}, {'id': 5, 'name': 'm0bwd_0j', 'supercategory': 'none'}]


In [35]:
coco_output = {
 "info": INFO,
 "licenses": LICENSES,
 "categories": CATEGORIES,
 "images": [],
 "annotations": []
}

In [36]:
IMAGE_DIR = '/content/train/myData2020/'
ANNOTATION_DIR = '/content/train/annotations/'

In [37]:
image_files = [f for f in listdir(IMAGE_DIR) if isfile(join(IMAGE_DIR, f))]
annotation_files = [f for f in listdir(ANNOTATION_DIR) if isfile(join(ANNOTATION_DIR, f))]

In [38]:
print(len(annotation_files))
print(len(image_files))

6331
2204


In [ ]:
image_id = 1
for image_filename in Tqdm(image_files):
    image = Image.open(IMAGE_DIR + '/' + image_filename)
    image_info = pycococreatortools.create_image_info(image_id, os.path.basename(image_filename), image.size)
    print(image_info)
    coco_output["images"].append(image_info)
    image_id = image_id + 1

In [40]:
segmentation_id = 1
for annotation_filename in Tqdm(annotation_files):
    image_id = [f for f in coco_output["images"] if stem(f['file_name'])==annotation_filename.split('_')[0]][0]['id']
    class_id = [x['id'] for x in CATEGORIES if x['name'] in annotation_filename][0]
    category_info = {'id': class_id, 'is_crowd': 'crowd' in image_filename}
    binary_mask = np.asarray(Image.open(f'{ANNOTATION_DIR}/{annotation_filename}').convert('1')).astype(np.uint8)
 
    annotation_info = pycococreatortools.create_annotation_info(segmentation_id, image_id, category_info, binary_mask, image.size, tolerance=2)

    if annotation_info is not None:
        coco_output["annotations"].append(annotation_info)
        segmentation_id = segmentation_id + 1

100%|██████████| 6331/6331 [06:56<00:00, 15.19it/s]


In [41]:
coco_output['categories'] = [{'id': id+1, 'name': clss_name, 'supercategory': 'none'} for id,(_,(name, clss_name)) in enumerate(classes.iterrows())]

In [43]:
import json
with open('/content/drive/MyDrive/images.json', 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

## Training the Model

In [44]:
!nvidia-smi

Mon Oct 18 13:02:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [45]:
torch.__version__

'1.9.0+cu111'

In [ ]:
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
!pip install pyyaml==5.1 pycocotools>=2.0.1

In [48]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer

In [49]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("dataset_train", {}, "images.json", "/content/train/myData2020")

In [53]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-\
InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025 
cfg.SOLVER.MAX_ITER = 2700
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(classes)

In [54]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

2021-10-18  13:56:33 INFO Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2): Sequent

In [55]:
!cp output/model_final.pth output/trained_model.pth